## Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import PIL
import pandas as pd
import tensorflow as tf

2023-11-13 14:38:19.524693: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Loading the data

In [2]:
TRAIN_DIR = "./imgs/train/"
img_height = 480
img_width = 640
batch_size = 32
training_dataset = tf.keras.utils.image_dataset_from_directory('./imgs/train/',
                                                      validation_split=0.2,
                                                        subset="training",
                                                        seed=123,
                                                        image_size=(img_height, img_width),
                                                        batch_size=batch_size)
validation_dataset = tf.keras.utils.image_dataset_from_directory('./imgs/train/',
                                                      validation_split=0.2,
                                                        subset="validation",
                                                        seed=123,
                                                        image_size=(img_height, img_width),
                                                        batch_size=batch_size)

Found 22424 files belonging to 10 classes.
Using 17940 files for training.


2023-11-13 14:40:07.394046: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Found 22424 files belonging to 10 classes.
Using 4484 files for validation.


In [4]:
# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
training_dataset = training_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

## A basic Keras model

In [ ]:
num_classes = len(training_dataset.class_names)

model = tf.keras.layers.Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])